Welcome to the **[30 Days of ML competition](https://www.kaggle.com/c/30-days-of-ml/overview)**!  In this notebook, you'll learn how to make your first submission.

Before getting started, make your own editable copy of this notebook by clicking on the **Copy and Edit** button.

# Step 1: Import helpful libraries

We begin by importing the libraries we'll need.  Some of them will be familiar from the **[Intro to Machine Learning](https://www.kaggle.com/learn/intro-to-machine-learning)** course and the **[Intermediate Machine Learning](https://www.kaggle.com/learn/intermediate-machine-learning)** course.

### This version of the notebook will follow the tutorials published by Abishek
https://www.kaggle.com/abhishek/

* part-1: Baseline

### Tensorflow
Including GPU support - sometimes - having trouble keeping tf-gpu working in Anaconda on Windoze


In [1]:
import tensorflow as tf
print(tf.__version__)

config = tf.compat.v1.ConfigProto(
    gpu_options=tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8),
    device_count={'GPU': 1},
    # session = tf.compat.v1.Session(config=config) 
    # tf.compat.v1.keras.backend.set_session(session)
)

session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


2.3.0
Num GPUs Available:  0


### Import the necessary modules

In [2]:
import time
import os
import numpy as np
import pandas as pd

# modules called out in part-1
from sklearn import model_selection
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor


# other modules from previous work
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import statsmodels.api as sm
from sklearn.feature_selection import RFE
from sklearn.metrics import confusion_matrix

# Regressors
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline, Pipeline

# visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# suppress "torch" warning in TPOT
import warnings
warnings.filterwarnings('ignore')


# Step 2: Load the data

Next, we'll load the training and test data.  

We set `index_col=0` in the code cell below to use the `id` column to index the DataFrame.  (*If you're not sure how this works, try temporarily removing `index_col=0` and see how it changes the result.*)

In [3]:
# Load the training data
path = ""  # "../input/30-days-of-ml/"

df_test = pd.read_csv(f"{path}test.csv")
sample_submission = pd.read_csv(f"{path}sample_submission.csv")


# Step 3: Prepare the data

Next, we'll need to handle the categorical columns (`cat0`, `cat1`, ... `cat9`).  

In the **[Categorical Variables lesson](https://www.kaggle.com/alexisbcook/categorical-variables)** in the Intermediate Machine Learning course, you learned several different ways to encode categorical variables in a dataset.  In this notebook, we'll use ordinal encoding and save our encoded features as new variables `X` and `X_test`.

### Implement KFold techniques
posted in the discussions by KGM - Abishek Thakur


In [4]:
# create train_folds.csv if it does not exist
if not Path("train_folds.csv").is_file():
    df_train = pd.read_csv(f"{path}train.csv")
    df_train["kfold"] = -1

    kf = model_selection.KFold(
        n_splits=5,
        shuffle=True,
        random_state=42
    )

    for fold, (train_indicies, valid_indicies) in enumerate(kf.split(X=df_train)):
        df_train.loc[valid_indicies, "kfold"] = fold

    df_train.to_csv("train_folds.csv", index=False)

df_train = pd.read_csv(f"{path}train_folds.csv")


In [5]:
print(df_train.head)

<bound method NDFrame.head of             id cat0 cat1 cat2 cat3 cat4 cat5 cat6 cat7 cat8  ...     cont6  \
0            1    B    B    B    C    B    B    A    E    C  ...  0.160266   
1            2    B    B    A    A    B    D    A    F    A  ...  0.558922   
2            3    A    A    A    C    B    D    A    D    A  ...  0.375348   
3            4    B    B    A    C    B    D    A    E    C  ...  0.239061   
4            6    A    A    A    C    B    D    A    E    A  ...  0.420667   
...        ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...       ...   
299995  499993    B    B    A    A    B    D    A    E    A  ...  0.450538   
299996  499996    A    B    A    C    B    B    A    E    E  ...  0.508502   
299997  499997    B    B    A    C    B    C    A    E    G  ...  0.372425   
299998  499998    A    B    A    C    B    B    A    E    E  ...  0.424243   
299999  499999    A    A    A    C    A    D    A    E    A  ...  0.328669   

           cont7     cont8     co

??? Why are we stripping test to only the catX columns ???

In [6]:
useful_features = [c for c in df_train.columns if c not in ("id", "target", "kfold")]

object_cols = [col for col in useful_features if 'cat' in col]

df_test = df_test[useful_features]

In [7]:
# Separate target from features
y = df_train['target']
features = df_train.drop(['target'], axis=1)

# Preview features
features.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,kfold
0,1,B,B,B,C,B,B,A,E,C,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,0
1,2,B,B,A,A,B,D,A,F,A,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,2
2,3,A,A,A,C,B,D,A,D,A,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,4
3,4,B,B,A,C,B,D,A,E,C,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,3
4,6,A,A,A,C,B,D,A,E,A,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,1


# Step 4: Train a model

Now that the data is prepared, the next step is to train a model.  

If you took the **[Intro to Machine Learning](https://www.kaggle.com/learn/intro-to-machine-learning)** courses, then you learned about **[Random Forests](https://www.kaggle.com/dansbecker/random-forests)**.  In the code cell below, we fit a random forest model to the data.

In [10]:
final_predictions = []
results = []

for fold in range(5):
    x_train = df_train[df_train.kfold != fold].reset_index(drop=True)
    x_valid = df_train[df_train.kfold == fold].reset_index(drop=True)
    x_test = df_test.copy()

    y_train = x_train.target
    y_valid = x_valid.target

    x_train = x_train[useful_features]
    x_valid = x_valid[useful_features]

    ordinal_encoder = OrdinalEncoder()
    x_train[object_cols] = ordinal_encoder.fit_transform(x_train[object_cols])
    x_valid[object_cols] = ordinal_encoder.transform(x_valid[object_cols])
    x_test[object_cols] = ordinal_encoder.transform(x_test[object_cols])

    xgb_params_tutorial = {
        'random_state': fold,
        'n_jobs': -1,
    }

    xgb_params_from_gridsearch = {
        'n_estimators': 5000,
        'learning_rate': 0.05,
        'subsample': 0.8,
        'colsample_bytree': 0.2,
        'max_depth': 3,
        'booster': 'gbtree',
        'reg_lambda': 0.2,
        'reg_alpha': 15,
        'random_state': fold,
        'n_jobs': -1,
        # 'gpu_id': 0,
        # 'tree_method': 'gpu_hist',
        # 'predictor': 'gpu_predictor'
    }

    # xgb_model = XGBRegressor(**xgb_params_tutorial)
    xgb_model = XGBRegressor(**xgb_params_from_gridsearch)

    %time xgb_model.fit(x_train, y_train)

    preds_valid = xgb_model.predict(x_valid)
    test_preds = xgb_model.predict(x_test)
    final_predictions.append(test_preds)

    mae = mean_squared_error(y_valid, preds_valid, squared=False)
    results.append(mae)

    print(fold, mae)

print(f"Average MAE: {sum(results)/len(results) :0.6f}")

Wall time: 3min 15s
0 0.7157593224363182
Wall time: 3min 19s
1 0.7160153248020477
Wall time: 3min 23s
2 0.7179753915186063
Wall time: 3min 24s
3 0.7177027838963904
Wall time: 3min 24s
4 0.7159340302614421


### Current best score
    Local: 0.71668 (5-fold)
    Kaggle-test: 0.71787 (V5 10-fold)
    Kaggle-result: 0.71853 (V5 10-fold)

Tutorial run as published results in:

    0  0.7242812912900478
    1  0.7232810321072864
    2  0.725452249623988
    3  0.725286377838993
    4  0.7242629367174096
    avg ~= 0.725
    
Upgrading to use settings from GridSearchCV work:

    Local: 0.71668 (5-fold)
    Kaggle-test: 0.71787 (V5 10-fold)
    Kagle-result: 0.71853 (V5 10-fold)

In [11]:
preds = np.mean(np.column_stack(final_predictions), axis=1)

In [ ]:
# sample_submission.target = preds
# sample_submission.to_csv("submission.csv", index=False)

# Step 5: Submit to the competition

We'll begin by using the trained model to generate predictions, which we'll save to a CSV file.

In [ ]:
# Use the model to generate predictions
predictions = model.predict(X_test)

# Save the predictions to a CSV file
output = pd.DataFrame({'Id': X_test.index,
                       'target': predictions})

output.to_csv('submission.csv', index=False)